In [ ]:

!pip install pandas numpy matplotlib seaborn plotly scipy statsmodels scikit-learn xgboost prophet shap -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

In [2]:

from google.colab import drive
drive.mount('/content/drive')
historical_data_path = '/content/drive/MyDrive/ds_mukul/csv_files/historical_data.csv'
sentiment_data_path = '/content/drive/MyDrive/ds_mukul/csv_files/fear_greed_index.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load datasets
print("Loading historical trading data...")
df_trades = pd.read_csv(historical_data_path, delimiter='|')
print(f"Loaded {len(df_trades):,} trading records")

print("\nLoading Fear & Greed sentiment data...")
df_sentiment = pd.read_csv(sentiment_data_path)
print(f"Loaded {len(df_sentiment):,} sentiment records")

print("\n Data loading complete!")

Loading historical trading data...
Loaded 211,224 trading records

Loading Fear & Greed sentiment data...
Loaded 365 sentiment records

✅ Data loading complete!


In [ ]:

print("🔍 HISTORICAL TRADING DATA OVERVIEW")
print("=" * 50)
print(f"Shape: {df_trades.shape}")
print(f"\nColumns: {list(df_trades.columns)}")
print(f"\nData types:")
print(df_trades.dtypes)
print(f"\nFirst few rows:")
display(df_trades.head())

🔍 HISTORICAL TRADING DATA OVERVIEW
Shape: (211224, 1)

Columns: ['Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp']

Data types:
Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp    object
dtype: object

First few rows:


,"Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp"
0,"0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@10..."
1,"0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@10..."
2,"0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@10..."
3,"0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@10..."
4,"0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@10..."


In [ ]:

print("🔍 SENTIMENT DATA OVERVIEW")
print("=" * 50)
print(f"Shape: {df_sentiment.shape}")
print(f"\nColumns: {list(df_sentiment.columns)}")
print(f"\nSentiment distribution:")
print(df_sentiment['Classification'].value_counts())
print(f"\nFirst few rows:")
display(df_sentiment.head())

🔍 SENTIMENT DATA OVERVIEW
Shape: (365, 2)

Columns: ['Date', 'Classification']

Sentiment distribution:
Classification
Greed            238
Fear              63
Extreme Greed     49
Extreme Fear      15
Name: count, dtype: int64

First few rows:


,Date,Classification
0,03-10-2025,Greed
1,02-10-2025,Greed
2,01-10-2025,Greed
3,30-09-2025,Greed
4,29-09-2025,Greed


In [6]:
# Clean sentiment data
print("🧹 Cleaning sentiment data...")

# Parse sentiment dates
df_sentiment['Date_parsed'] = pd.to_datetime(df_sentiment['Date'], format='%d-%m-%Y', errors='coerce')
df_sentiment['Date'] = df_sentiment['Date_parsed'].dt.date

# Create sentiment score (numerical encoding)
sentiment_mapping = {
    'Extreme Fear': 1,
    'Fear': 2,
    'Neutral': 3,
    'Greed': 4,
    'Extreme Greed': 5
}
df_sentiment['Sentiment_Score'] = df_sentiment['Classification'].map(sentiment_mapping)

print(f"Cleaned {len(df_sentiment)} sentiment records")
print(f"Date range: {df_sentiment['Date'].min()} to {df_sentiment['Date'].max()}")

🧹 Cleaning sentiment data...
Cleaned 365 sentiment records
Date range: 2024-10-03 to 2025-10-03


In [ ]:

print("Loading historical trading data...")
df_trades = pd.read_csv(historical_data_path)
print(f"Loaded {len(df_trades):,} trading records")

# Clean timestamps
df_trades['Timestamp_parsed'] = pd.to_datetime(df_trades['Timestamp IST'],
                                               format='%d-%m-%Y %H:%M',
                                               errors='coerce')
df_trades['Date'] = df_trades['Timestamp_parsed'].dt.date

# Convert numeric columns
numeric_columns = ['Execution Price', 'Size Tokens', 'Size USD', 'Closed PnL', 'Fee']
for col in numeric_columns:
    if col in df_trades.columns:
        df_trades[col] = pd.to_numeric(df_trades[col], errors='coerce')

print(f"Cleaned {len(df_trades)} records")
print(f"Date range: {df_trades['Date'].min()} to {df_trades['Date'].max()}")



Loading historical trading data...
Loaded 211,224 trading records
Cleaned 211224 records
Date range: 2023-05-01 to 2025-05-01


In [ ]:

print("📊 Aggregating daily trading metrics...")

daily_metrics = df_trades.groupby('Date').agg({
    'Size USD': ['sum', 'mean', 'count'],
    'Closed PnL': ['sum', 'mean', 'std'],
    'Fee': 'sum',
    'Side': lambda x: (x == 'BUY').sum() / len(x),  # Buy ratio
    'Execution Price': ['mean', 'std']
}).reset_index()

# Flatten column names
daily_metrics.columns = ['Date', 'Total_Volume_USD', 'Avg_Trade_Size_USD', 'Trade_Count',
                        'Total_PnL', 'Avg_PnL', 'PnL_Volatility', 'Total_Fees',
                        'Buy_Ratio', 'Avg_Price', 'Price_Volatility']

# Calculate additional metrics
daily_metrics['Net_PnL'] = daily_metrics['Total_PnL'] - daily_metrics['Total_Fees']
daily_metrics['Win_Rate'] = daily_metrics['Total_PnL'] > 0
daily_metrics['ROI_Percent'] = (daily_metrics['Net_PnL'] / daily_metrics['Total_Volume_USD']) * 100

print(f"Created daily metrics for {len(daily_metrics)} days")
display(daily_metrics.head())

📊 Aggregating daily trading metrics...
Created daily metrics for 480 days


,Date,Total_Volume_USD,Avg_Trade_Size_USD,Trade_Count,Total_PnL,Avg_PnL,PnL_Volatility,Total_Fees,Buy_Ratio,Avg_Price,Price_Volatility,Net_PnL,Win_Rate,ROI_Percent
0,2023-05-01,477.00,159.000000,3,0.000000,0.000000,0.000000,0.000000,1.000000,1898.133333,0.404145,0.000000,False,0.000000
1,2023-12-05,50005.83,5556.203333,9,0.000000,0.000000,0.000000,12.501455,0.777778,11038.300000,17477.946554,-12.501455,False,-0.025000
2,2023-12-14,113203.35,10291.213636,11,-205.434737,-18.675885,179.663534,28.300831,0.454545,8031.868818,17176.671271,-233.735568,False,-0.206474
3,2023-12-15,10609.95,5304.975000,2,-24.632034,-12.316017,4.474304,2.652489,1.000000,2.982000,0.000000,-27.284523,False,-0.257160
4,2023-12-16,15348.77,5116.256667,3,0.000000,0.000000,0.000000,3.837189,1.000000,0.384707,0.000029,-3.837189,False,-0.025000


In [9]:
# Merge daily trading metrics with sentiment data
print("🔗 Merging trading data with sentiment...")

merged_data = pd.merge(daily_metrics,
                      df_sentiment[['Date', 'Classification', 'Sentiment_Score']],
                      on='Date',
                      how='inner')

print(f"Merged dataset contains {len(merged_data)} days")
print(f"\nSentiment distribution in merged data:")
print(merged_data['Classification'].value_counts())

display(merged_data.head())

🔗 Merging trading data with sentiment...
Merged dataset contains 209 days

Sentiment distribution in merged data:
Classification
Greed            97
Fear             50
Extreme Greed    47
Extreme Fear     15
Name: count, dtype: int64


,Date,Total_Volume_USD,Avg_Trade_Size_USD,Trade_Count,Total_PnL,Avg_PnL,PnL_Volatility,Total_Fees,Buy_Ratio,Avg_Price,Price_Volatility,Net_PnL,Win_Rate,ROI_Percent,Classification,Sentiment_Score
0,2024-10-03,132935.69,14770.632222,9,100.375782,11.152865,20.273575,46.527491,0.666667,26799.238067,31775.921226,53.848291,True,0.040507,Fear,2
1,2024-10-04,0.34,0.340000,1,0.000000,0.000000,NaN,0.000000,0.000000,0.001100,NaN,0.000000,False,0.000000,Fear,2
2,2024-10-05,605605.74,16367.722703,37,5767.161892,155.869240,217.087973,211.961771,0.243243,60568.594602,10235.086311,5555.200121,True,0.917296,Greed,4
3,2024-10-06,300130.56,13642.298182,22,130.487079,5.931231,17.741124,105.045693,0.272727,62166.409091,324.918694,25.441386,True,0.008477,Greed,4
4,2024-10-07,219984.65,10475.459524,21,0.000000,0.000000,0.000000,76.994616,0.000000,63684.428571,182.244773,-76.994616,False,-0.035000,Greed,4


In [10]:
# Summary statistics by sentiment
print("📈 SUMMARY STATISTICS BY SENTIMENT")
print("=" * 60)

summary_by_sentiment = merged_data.groupby('Classification').agg({
    'Total_Volume_USD': ['mean', 'std'],
    'Total_PnL': ['mean', 'std'],
    'ROI_Percent': ['mean', 'std'],
    'Trade_Count': ['mean', 'std'],
    'Buy_Ratio': ['mean', 'std']
}).round(4)

display(summary_by_sentiment)

📈 SUMMARY STATISTICS BY SENTIMENT


Total_Volume_USD                 Total_PnL               \
                           mean           std        mean          std   
Classification                                                           
Extreme Fear       1.070249e+07  7.290153e+06  72326.1547  104206.8109   
Extreme Greed      1.871502e+06  1.891134e+06  48104.7460  107437.9456   
Fear               9.336493e+06  1.215102e+07  71187.4226  117922.7025   
Greed              4.136674e+06  7.699224e+06  30568.9735   89204.2009   

               ROI_Percent         Trade_Count            Buy_Ratio          
                      mean     std        mean        std      mean     std  
Classification                                                               
Extreme Fear        1.0511  1.8532   2072.8000  1580.7979    0.4966  0.1407  
Extreme Greed       2.7681  6.6553    613.8511   522.7895    0.4683  0.1667  
Fear                1.0958  1.5506   1138.0600   806.2010    0.4852  0.1925  
Greed               1.2694  2.8567    810.9897  1085.1693    0.4789  0.2242

In [11]:
!pip install -U kaleido -
!pip install -U kaleido==0.2.1

ERROR: Invalid requirement: '-': Expected package name at the start of dependency specifier
    -
    ^
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/__init__.py", line 3, in <module>
    from pip._internal.utils import _log
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/utils/_log.py", line 8, in <module>
    import logging
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1128, in get_code
  File "<frozen importlib._bootstrap_external>", line 757, in _compile_bytecode
KeyboardInterrupt
^C


In [16]:
import os

output_dir = '/content/drive/MyDrive/ds_mukul/outputs/'
os.makedirs(output_dir, exist_ok=True)

print(f"Created directory: {output_dir}")

Created directory: /content/drive/MyDrive/ds_mukul/outputs/


In [ ]:

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Daily Volume by Sentiment', 'PnL Distribution by Sentiment',
                   'ROI Distribution by Sentiment', 'Trading Activity Overview'),
    specs=[[{"secondary_y": False}, {"type": "box"}],
           [{"type": "violin"}, {"secondary_y": False}]]
)

# Row 1, Col 1
volume_data = merged_data.sort_values('Date')
fig.add_trace(
    go.Scatter(x=volume_data['Date'],
               y=volume_data['Total_Volume_USD'],
               mode='markers',
               marker=dict(color=volume_data['Sentiment_Score'],
                          colorscale='RdYlGn',
                          colorbar=dict(title="Sentiment Score")),
               name='Volume vs Sentiment'),
    row=1, col=1
)

# Row 1, Col 2
for sentiment in merged_data['Classification'].unique():
    sentiment_data = merged_data[merged_data['Classification'] == sentiment]
    fig.add_trace(
        go.Box(y=sentiment_data['Total_PnL'],
               name=sentiment),
        row=1, col=2
    )

# Row 2, Col 1
for sentiment in merged_data['Classification'].unique():
    sentiment_data = merged_data[merged_data['Classification'] == sentiment]
    fig.add_trace(
        go.Violin(y=sentiment_data['ROI_Percent'],
                  name=f'{sentiment} ROI',
                  showlegend=False),
        row=2, col=1
    )

# Row 2, Col 2
fig.add_trace(
    go.Scatter(x=merged_data['Trade_Count'],
               y=merged_data['Buy_Ratio'],
               mode='markers',
               marker=dict(color=merged_data['Sentiment_Score'],
                          colorscale='RdYlGn',
                          size=8),
               name='Activity Pattern'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Trading Behavior vs Market Sentiment Analysis")
fig.show()


fig.write_image("/content/drive/MyDrive/ds_mukul/outputs/comprehensive_analysis.png",
                width=1920, height=1080, scale=2)

print("✅ Visualization saved to outputs folder!")

✅ Visualization saved to outputs folder!


## 7. Statistical Analysis

In [13]:
from scipy import stats
from statsmodels.stats.contingency_tables import mcnemar

print("📊 STATISTICAL SIGNIFICANCE TESTING")
print("=" * 50)

# Test if PnL differs significantly across sentiment regimes
fear_pnl = merged_data[merged_data['Classification'] == 'Fear']['Total_PnL']
greed_pnl = merged_data[merged_data['Classification'] == 'Greed']['Total_PnL']

# T-test
t_stat, p_value = stats.ttest_ind(fear_pnl, greed_pnl, equal_var=False)
print(f"\nPnL Comparison: Fear vs Greed")
print(f"Fear PnL: {fear_pnl.mean():.2f} ± {fear_pnl.std():.2f}")
print(f"Greed PnL: {greed_pnl.mean():.2f} ± {greed_pnl.std():.2f}")
print(f"T-statistic: {t_stat:.4f}, P-value: {p_value:.6f}")
print(f"Significant: {'Yes' if p_value < 0.05 else 'No'}")

# Correlation analysis
correlation = merged_data[['Sentiment_Score', 'Total_PnL', 'ROI_Percent',
                          'Trade_Count', 'Buy_Ratio']].corr()
print(f"\nCorrelation Matrix:")
display(correlation)

📊 STATISTICAL SIGNIFICANCE TESTING

PnL Comparison: Fear vs Greed
Fear PnL: 71187.42 ± 117922.70
Greed PnL: 30568.97 ± 89204.20
T-statistic: 2.1403, P-value: 0.035425
Significant: Yes

Correlation Matrix:


,Sentiment_Score,Total_PnL,ROI_Percent,Trade_Count,Buy_Ratio
Sentiment_Score,1.000000,-0.128338,0.122940,-0.318136,-0.035834
Total_PnL,-0.128338,1.000000,0.532509,0.240448,-0.111274
ROI_Percent,0.122940,0.532509,1.000000,-0.026779,-0.164124
Trade_Count,-0.318136,0.240448,-0.026779,1.000000,0.034576
Buy_Ratio,-0.035834,-0.111274,-0.164124,0.034576,1.000000


## 8. Feature Engineering for Modeling

In [14]:
# Create lagged features and technical indicators
print("⚙️ Engineering features for modeling...")

# Sort by date
merged_data = merged_data.sort_values('Date').reset_index(drop=True)

# Create lagged sentiment features
for lag in [1, 3, 7]:
    merged_data[f'Sentiment_Lag_{lag}'] = merged_data['Sentiment_Score'].shift(lag)

# Rolling statistics
for window in [3, 7, 14]:
    merged_data[f'PnL_MA_{window}'] = merged_data['Total_PnL'].rolling(window).mean()
    merged_data[f'Volume_MA_{window}'] = merged_data['Total_Volume_USD'].rolling(window).mean()
    merged_data[f'Volatility_{window}'] = merged_data['Total_PnL'].rolling(window).std()

# Target variables for modeling
merged_data['Profitable_Day'] = (merged_data['Total_PnL'] > 0).astype(int)
merged_data['High_Volume_Day'] = (merged_data['Total_Volume_USD'] >
                                 merged_data['Total_Volume_USD'].median()).astype(int)

# Save engineered dataset
engineered_data = merged_data.dropna()
engineered_data.to_csv('/content/drive/MyDrive/ds_mukul/csv_files/engineered_dataset.csv', index=False)

print(f"✅ Feature engineering complete!")
print(f"Engineered dataset shape: {engineered_data.shape}")
print(f"New features created: {len(engineered_data.columns) - len(merged_data.columns)} additional columns")

⚙️ Engineering features for modeling...
✅ Feature engineering complete!
Engineered dataset shape: (196, 30)
New features created: 0 additional columns


## 9. Key Insights Summary

In [15]:
print("🎯 KEY INSIGHTS FROM EDA")
print("=" * 40)

insights = []

# Calculate key metrics for insights
fear_roi = merged_data[merged_data['Classification'] == 'Fear']['ROI_Percent'].mean()
greed_roi = merged_data[merged_data['Classification'] == 'Greed']['ROI_Percent'].mean()

insights.append(f"📊 ROI Performance: Fear periods show {fear_roi:.2f}% avg ROI vs Greed periods {greed_roi:.2f}% avg ROI")

corr_sentiment_pnl = merged_data['Sentiment_Score'].corr(merged_data['Total_PnL'])
insights.append(f"📈 Sentiment-PnL Correlation: {corr_sentiment_pnl:.4f} ({'Strong' if abs(corr_sentiment_pnl) > 0.5 else 'Moderate' if abs(corr_sentiment_pnl) > 0.3 else 'Weak'})")

profitable_days = (merged_data['Total_PnL'] > 0).mean() * 100
insights.append(f"💰 Win Rate: {profitable_days:.1f}% of trading days were profitable")

for insight in insights:
    print(insight)

print("\n📋 Next Steps:")
print("• Build predictive models in Notebook 2")
print("• Test trading strategies based on sentiment signals")
print("• Validate model performance with walk-forward analysis")

🎯 KEY INSIGHTS FROM EDA
📊 ROI Performance: Fear periods show 1.10% avg ROI vs Greed periods 1.27% avg ROI
📈 Sentiment-PnL Correlation: -0.1283 (Weak)
💰 Win Rate: 86.6% of trading days were profitable

📋 Next Steps:
• Build predictive models in Notebook 2
• Test trading strategies based on sentiment signals
• Validate model performance with walk-forward analysis
